In [60]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import numpy as np

diabet_data = pd.read_csv('data/diabetes_data.csv')

clean_diabet_data = diabet_data.drop_duplicates()

def low_informative_feature(data):
    low_information_cols = []
    for col in data.columns:
        top_freq = data[col].value_counts(normalize=True).max()
        nunique_ratio = data[col].nunique() / data[col].count()
        if top_freq > 0.95:
            low_information_cols.append(col)
        if nunique_ratio > 0.95:
            low_information_cols.append(col)
    return low_information_cols
low_information_cols = low_informative_feature(clean_diabet_data)

clean_diabet_data = clean_diabet_data.drop('Gender', axis=1)

null_columns = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

clean_diabet_data[null_columns] = clean_diabet_data[null_columns].applymap(lambda x: np.nan if x == 0 else x)

for col in null_columns:
    if clean_diabet_data[col].isnull().sum() / len(clean_diabet_data[col]) > .3:
        clean_diabet_data = clean_diabet_data.drop(col, axis=1)


clean_diabet_data = clean_diabet_data.dropna(thresh=6)

clean_diabet_data = clean_diabet_data.fillna(clean_diabet_data.median())

def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

for col in clean_diabet_data.columns:
    outliers, cleaned = outliers_iqr_mod(clean_diabet_data, col)
    
    print(col, outliers.shape[0])




Pregnancies 4
Glucose 0
BloodPressure 14
SkinThickness 87
BMI 8
DiabetesPedigreeFunction 29
Age 9
Outcome 0
